In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
# timesteps from 1h to 24h 
df = pd.read_csv("cleaned_data\\24timestep1h.csv", parse_dates=['tstp'])
df = df.drop(['date'], axis=1) # date column when joined with the holiday

In [3]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

def create_fourier(date_index):
    fourier = CalendarFourier(
        freq='D',
        order=1
    )
    dp = DeterministicProcess(
        index=date_index,
        constant=True,
        order=1,
        seasonal=True,
        additional_terms=[fourier],
        drop=True
    )

    return dp

In [4]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# label encoding
precip_encoder = OneHotEncoder(sparse_output=False)
holiday_encoder = OneHotEncoder(sparse_output=False)
summary_encoder = OneHotEncoder(sparse_output=False)
household_encoder = LabelEncoder()
stdor_encoder = OneHotEncoder(sparse_output=False)
acorn_encoder = LabelEncoder()

precip_labels = precip_encoder.fit_transform(df[['precipType']])
holiday_labels = holiday_encoder.fit_transform(df[['Type']])
summary_labels = summary_encoder.fit_transform(df[['summary']])
household_labels = household_encoder.fit_transform(df[['LCLid']])
stdor_labels = stdor_encoder.fit_transform(df[['stdorToU']])
acorn_labels = acorn_encoder.fit_transform(df[['Acorn']])

precip_labels = pd.DataFrame(precip_labels, columns=precip_encoder.get_feature_names_out()).astype('int8')
holiday_labels = pd.DataFrame(holiday_labels, columns=holiday_encoder.get_feature_names_out()).astype('int8')
summary_labels = pd.DataFrame(summary_labels, columns=summary_encoder.get_feature_names_out()).astype('int8')
household_labels = pd.DataFrame(household_labels, columns=['household_label']).astype('int8')
stdor_labels = pd.DataFrame(stdor_labels, columns=stdor_encoder.get_feature_names_out()).astype('int8')
acorn_labels = pd.DataFrame(acorn_labels, columns=['acorn_label']).astype('int8')

encoded_df = pd.concat([df, precip_labels, holiday_labels, summary_labels, household_labels, stdor_labels, acorn_labels], axis=1)
encoded_df = encoded_df.drop(['precipType', 'Type', 'summary', 'LCLid', 'stdorToU', 'Acorn'], axis=1)

c:\Users\Usrer\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Usrer\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
# process fourier features for each group and saved each group's deterministic process
import math

dps = {}

all_household_ids = encoded_df['household_label'].unique()
households_grp = encoded_df.groupby('household_label')
training_df = []
testing_df = []

for household_id in all_household_ids:
    # get the household from the entire df
    print("Household id:", household_id)
    singlehousehold_df = households_grp.get_group(household_id).sort_values(by='tstp', ascending=True)

    # split into train, test
    cutoff_point = math.floor(singlehousehold_df.shape[0] * 0.9)
    training_singlehousehold_df = singlehousehold_df.iloc[:cutoff_point].reset_index()
    testing_singlehousehold_df = singlehousehold_df.iloc[cutoff_point:].reset_index()

    # create fourier features
    fourier_obj = create_fourier(training_singlehousehold_df.set_index('tstp').asfreq('h').index)
    fourier_train_features = fourier_obj.in_sample().reset_index().drop(['tstp'], axis=1)
    training_singlehousehold_df = pd.concat([training_singlehousehold_df, fourier_train_features], axis=1)
    fourier_test_features = fourier_obj.out_of_sample(testing_singlehousehold_df.shape[0]).reset_index().drop(['index'], axis=1)
    testing_singlehousehold_df = pd.concat([testing_singlehousehold_df.reset_index().drop(['index'], axis=1), fourier_test_features], axis=1)
    
    # save the dp object
    dps[household_id] = fourier_obj

    # append to the household_dfs list
    training_df.append(training_singlehousehold_df)
    testing_df.append(testing_singlehousehold_df)

training_df = pd.concat(training_df, axis=0)
testing_df = pd.concat(testing_df, axis=0)

Household id: 35
Household id: 30
Household id: 14
Household id: 46
Household id: 41
Household id: 24
Household id: 26
Household id: 37
Household id: 6
Household id: 45
Household id: 19
Household id: 3
Household id: 39
Household id: 32
Household id: 8
Household id: 22
Household id: 11
Household id: 38
Household id: 42
Household id: 10
Household id: 15
Household id: 16
Household id: 29
Household id: 36
Household id: 1
Household id: 23
Household id: 31
Household id: 28
Household id: 21
Household id: 7
Household id: 49
Household id: 17
Household id: 27
Household id: 0
Household id: 44
Household id: 5
Household id: 40
Household id: 12
Household id: 48
Household id: 33
Household id: 13
Household id: 2
Household id: 50
Household id: 43
Household id: 25
Household id: 34
Household id: 20
Household id: 18
Household id: 4
Household id: 9
Household id: 47


In [6]:
# scale the values
from sklearn.preprocessing import MinMaxScaler

# drop unnecessary columns
training_df = training_df.drop(['index', 'tstp'], axis=1)
testing_df = testing_df.drop(['level_0', 'tstp'], axis=1)

# separate into features and target
X_train = training_df.drop(['energy(kWh/hh)'], axis=1)
X_test = testing_df.drop(['energy(kWh/hh)'], axis=1)
y_train = training_df['energy(kWh/hh)']
y_test = testing_df['energy(kWh/hh)']

# store the household ids for future evaluation
test_household_ids = X_test['household_label']
train_household_ids = X_train['household_label']

# Scale with MinMax Normalization
scaler = MinMaxScaler(feature_range=(0, 1)) # scale to 0 and 1
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# reshape for LSTM, CNN-LSTM models
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

print(X_train.shape)
print(X_test.shape)

(686587, 1, 134)
(76315, 1, 134)


In [7]:
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score

def adjusted_r2(y_pred, y_test):
   p = X_train.shape[2]
   r2 = r2_score(y_pred, y_test)
   n = y_test.shape[0]
   
   adjusted_coefficient = 1 - (1 - r2) * (n - 1) / (n - p - 1)

   return adjusted_coefficient

def evaluate_model(y_pred, y_test, deep_learning):
  """
    Evaluate the model predictions with all the regression metrics
  """
  if deep_learning:
    prediction = y_pred.ravel()

  else:
    prediction = y_pred

  print("RMSE:", mean_squared_error(prediction, y_test, squared=False))
  print("MSE:", mean_squared_error(prediction, y_test, squared=True))
  print("MAE:", mean_absolute_error(prediction, y_test))
  print("MAPE:", mean_absolute_percentage_error(prediction, y_test))
  print("R2 Score:", r2_score(prediction, y_test))
  print("Adjusted R2 Score:", adjusted_r2(prediction, y_test))

def plot_forecast(y_pred, y_test, number_of_slices=3, steps=168, skip=0):
    """
      Plot the forecasted values against the truth values for LSTM and CNN-LSTM
    """
    fig, axes = plt.subplots(number_of_slices, 1, sharey=True, figsize=(14, 10))
    timesteps = np.arange(0, steps)
    
    for i in range(number_of_slices):
        start_idx = 0 + (steps * i) + (skip)
        end_idx = steps * (1 + i) + (skip)
        sampled_test = y_test.iloc[start_idx:end_idx]
        sampled_pred = y_pred.reshape(-1)[start_idx:end_idx]
        mape_score = mean_absolute_percentage_error(sampled_test, sampled_pred)
        
        sns.lineplot(x=timesteps, y=y_test.iloc[start_idx:end_idx], label="truth-value", marker='o', alpha=0.3, ax=axes[i])
        sns.lineplot(x=timesteps, y=y_pred.reshape(-1)[start_idx:end_idx], label="pred-value", marker='o', ax=axes[i])
        axes[i].set_title(f'MAPE: {round(mape_score, 2)}')

    plt.tight_layout()
    plt.show()
    

### LSTM

In [8]:
# Train with the best hyperparameter set
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Dense, LSTM,
    TimeDistributed, Dropout, Input
)

# for reproducibility
tf.random.set_seed(42)
tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_mse', patience=5)

# input 
input_layer = Input(shape=(1, X_train.shape[2]))

# stacked lstm
lstm1 = LSTM(200, return_sequences=True)(input_layer)
lstm2 = LSTM(200, return_sequences=True)(lstm1)
dropout1 = Dropout(0.2)(lstm2)

# full connected layer
fc1 = TimeDistributed(Dense(168, activation='relu', kernel_regularizer='l1'))(dropout1)
fc2 = TimeDistributed(Dense(64, activation='relu'))(fc1)

# output
output_layer = TimeDistributed(Dense(1))(fc2)  
model = Model(inputs=input_layer, outputs=output_layer)


model.compile(
    loss=tf.keras.losses.MSE,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['mae', 'mse', 'mape']
)

history = model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_test, y_test),
    batch_size=168,
    callbacks=[earlystop]
)

Epoch 1/50
4087/4087 [==============================] - 36s 8ms/step - loss: 2.4158 - mae: 0.2067 - mse: 0.1395 - mape: 1375725.0000 - val_loss: 0.1523 - val_mae: 0.1827 - val_mse: 0.1256 - val_mape: 559695.2500
Epoch 2/50
4087/4087 [==============================] - 31s 7ms/step - loss: 0.0965 - mae: 0.1441 - mse: 0.0741 - mape: 785554.0625 - val_loss: 0.0598 - val_mae: 0.1053 - val_mse: 0.0310 - val_mape: 337159.8750
Epoch 3/50
4087/4087 [==============================] - 32s 8ms/step - loss: 0.0295 - mae: 0.0494 - mse: 0.0068 - mape: 516746.5312 - val_loss: 0.0214 - val_mae: 0.0383 - val_mse: 0.0043 - val_mape: 366523.4688
Epoch 4/50
4087/4087 [==============================] - 31s 8ms/step - loss: 0.0184 - mae: 0.0363 - mse: 0.0037 - mape: 421660.7500 - val_loss: 0.0162 - val_mae: 0.0277 - val_mse: 0.0038 - val_mape: 310152.8750
Epoch 5/50
4087/4087 [==============================] - 31s 8ms/step - loss: 0.0158 - mae: 0.0347 - mse: 0.0036 - mape: 390530.6250 - val_loss: 0.0186 - va

In [9]:
# export the model

model.save("models\\lstm_model.h5")

In [10]:
import tensorflow as tf

def pinball_loss(y, y_hat, alpha):
  """
    Loss function for the deep learning models
  """

  error = (y - y_hat)
  loss = tf.keras.backend.mean(
    tf.keras.backend.maximum(alpha * error, (alpha - 1) * error), 
    axis=-1
  )

  return loss

def plot_training_metrics(history):
  """
    Function to plot the training metrics of the deep learning models
  """
  training_metrics = history.history
  loss = training_metrics['loss']
  val_loss = training_metrics['val_loss']
  loss_20 = training_metrics['output_20_loss']
  val_loss_20 = training_metrics['val_output_20_loss']
  loss_50 = training_metrics['output_50_loss']
  val_loss_50 = training_metrics['val_output_50_loss']
  loss_80 = training_metrics['output_80_loss']
  val_loss_80 = training_metrics['val_output_80_loss']


  timesteps = np.arange(len(loss))
  fig, axes = plt.subplots(2, 2, sharey=True, figsize=(14, 10))

  sns.lineplot(x=timesteps, y=loss, label='loss', marker='o', ax=axes[0][0])
  sns.lineplot(x=timesteps, y=val_loss, label='val_loss', marker='o', ax=axes[0][0])
  axes[0][0].set_title("Loss vs Validation Loss")
  axes[0][0].set_xlabel("Epochs")  
  axes[0][0].set_ylabel("Overall Loss")

  sns.lineplot(x=timesteps, y=loss_20, label='loss_20', marker='o', ax=axes[0][1])
  sns.lineplot(x=timesteps, y=val_loss_20, label='val_loss_20', marker='o', ax=axes[0][1])
  axes[0][1].set_title("Loss vs Validation Loss; alpha=0.2")
  axes[0][1].set_xlabel("Epochs")  
  axes[0][1].set_ylabel("Pinball Loss")

  sns.lineplot(x=timesteps, y=loss_50, label='loss_50', marker='o', ax=axes[1][0])
  sns.lineplot(x=timesteps, y=val_loss_50, label='val_loss_50', marker='o', ax=axes[1][0])
  axes[1][0].set_title("Loss vs Validation Loss; alpha=0.5")
  axes[1][0].set_xlabel("Epochs")  
  axes[1][0].set_ylabel("Pinball Loss")

  sns.lineplot(x=timesteps, y=loss_80, label='loss_80', marker='o', ax=axes[1][1])
  sns.lineplot(x=timesteps, y=val_loss_80, label='val_loss_80', marker='o', ax=axes[1][1])
  axes[1][1].set_title("Loss vs Validation Loss; alpha=0.8")
  axes[1][1].set_xlabel("Epochs")  
  axes[1][1].set_ylabel("Pinball Loss")

  plt.tight_layout()
  plt.show()
  

In [13]:
# input 
input_layer = Input(shape=(1, X_train.shape[2]))

earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# stacked lstm
layer = LSTM(300, return_sequences=True)(input_layer)
layer = Dropout(0.2)(layer)

# full connected layer
layer = TimeDistributed(Dense(168, activation='relu', kernel_regularizer='l1'))(layer)
layer = TimeDistributed(Dense(64, activation='relu'))(layer)

# output
output_layer_20 = TimeDistributed(Dense(1), name="output_20")(layer)
output_layer_50 = TimeDistributed(Dense(1), name="output_50")(layer)
output_layer_80 = TimeDistributed(Dense(1), name="output_80")(layer)
model = Model(inputs=[input_layer], outputs=[output_layer_20, output_layer_50, output_layer_80])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss={
        "output_20": lambda y, y_hat: pinball_loss(y, y_hat, 0.2),
        "output_50": lambda y, y_hat: pinball_loss(y, y_hat, 0.5),
        "output_80": lambda y, y_hat: pinball_loss(y, y_hat, 0.8)
    }
)

history = model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_test, y_test),
    batch_size=168,
    callbacks=[earlystop]
)

Epoch 1/50
4087/4087 [==============================] - 37s 9ms/step - loss: 2.9377 - output_20_loss: 0.0599 - output_50_loss: 0.1074 - output_80_loss: 0.1064 - val_loss: 0.2908 - val_output_20_loss: 0.0647 - val_output_50_loss: 0.1065 - val_output_80_loss: 0.0959
Epoch 2/50
4087/4087 [==============================] - 30s 7ms/step - loss: 0.2436 - output_20_loss: 0.0534 - output_50_loss: 0.0882 - output_80_loss: 0.0801 - val_loss: 0.2561 - val_output_20_loss: 0.0599 - val_output_50_loss: 0.0940 - val_output_80_loss: 0.0825
Epoch 3/50
4087/4087 [==============================] - 30s 7ms/step - loss: 0.2258 - output_20_loss: 0.0512 - output_50_loss: 0.0821 - output_80_loss: 0.0733 - val_loss: 0.2387 - val_output_20_loss: 0.0569 - val_output_50_loss: 0.0871 - val_output_80_loss: 0.0760
Epoch 4/50
4087/4087 [==============================] - 29s 7ms/step - loss: 0.2131 - output_20_loss: 0.0493 - output_50_loss: 0.0771 - output_80_loss: 0.0678 - val_loss: 0.2236 - val_output_20_loss: 0.054

In [14]:
model.save('models\\quantile_lstm.h5')